In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import configargparse

parser = configargparse.ArgParser(default_config_files=['../configs/config.ini'])

parser.add('--config', is_config_file=True, help='Path to config file')
parser.add('--data-folder', type=str, required=True, help='Root data directory containing Images and CHM folders')
parser.add('--hdf5-file', type=str, default='Finland_CHM.h5', help='HDF5 file containing CHM dataset')
parser.add('--output-dir', type=str, default='output/chmpredict', help='Directory to save output models and logs')
parser.add('--learning-rate', type=float, default=1e-4, help='Learning rate for optimizer')
parser.add('--batch-size', type=int, default=16, help='Batch size for DataLoader')
parser.add('--epochs', type=int, default=50, help='Number of training epochs')
parser.add('--patience', type=int, default=5, help='Patience for early stopping')

config, _ = parser.parse_known_args()

config.data_folder = '/Users/anisr/Documents/Finland_CHM'
config.output_dir = '../output/chmpredict'
config.batch_size = 4

print(config)

In [ ]:
import os
import torch

from chmpredict.data.create import create_patch_pool
from chmpredict.data.loader import load_fn

from chmpredict.model.build import build_fn
from chmpredict.model.callback import EarlyStopping, ModelCheckpoint
from chmpredict.model.train import train_fn
from chmpredict.model.eval import eval_fn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(config.output_dir):
    os.makedirs(config.output_dir)

hdf5_path = os.path.join(config.data_folder, config.hdf5_file)

if not os.path.exists(hdf5_path):
    print(f"Creating patch pool at {hdf5_path}...")
    create_patch_pool(config.data_folder, config.hdf5_file, patch_size=256, stride=256)
else:
    print(f"HDF5 file {hdf5_path} already exists. Skipping creation.")

train_loader, val_loader, test_loader = load_fn(hdf5_path, config.batch_size)

model, criterion, optimizer = build_fn(config.learning_rate, config.output_dir, device)

early_stopping = EarlyStopping(patience=config.patience)
model_checkpoint = ModelCheckpoint(output_dir=config.output_dir)

In [ ]:

train_fn(train_loader, val_loader, model, criterion, optimizer, config.epochs, config.patience, config.output_dir, device, callbacks=[early_stopping, model_checkpoint])

eval_fn(test_loader, model, criterion, config.output_dir, device)